In [160]:
from scipy.spatial.distance import cosine
import os

from bs4 import BeautifulSoup
import numpy as np


import pymorphy2
import re
from stop_words import get_stop_words

AllowedWords = '[а-я]+'

wordNormal = pymorphy2.MorphAnalyzer()
stop_words = {wordNormal.normal_forms(i)[0] for i in get_stop_words('ru')}

docs1 = dict()
docs2 = dict()

MODEL_NAME = 'finaleModel.bin'


search = '''Как добавить виджет на страницу'''




#ToDo maybe not normal form?

#ToDo use unclean docs?

#ToDo ./sent2vec-master/fasttext ??

In [161]:
def cleanTest():
    for num,name in enumerate(os.listdir("./Dirty/Data/")):
        with open("./Dirty/Data/"+name, 'r') as doc:
            soup = BeautifulSoup(doc.read(), 'html.parser')
            with open("./Clean/Data/"+str(num)+'.txt', 'w') as clean:
                clean.write(' '.join(soup.getText(' ').strip().split()))



In [162]:
def createTestfromData():
    text = ''
    for name in os.listdir("./Clean/Data/"):
        try:
            with open("./Clean/Data/"+name, 'r',) as doc:
                #docs1[name] = ""
                for word in re.findall(AllowedWords, doc.read().lower()):
                    normal = wordNormal.normal_forms(word)[0]
                    #docs1[name] += str(normal)
                    #docs1[name] += ' '
                    text += str(normal) + ' '
                text += '\n'
        except:
            print("haha")
    with open('./Technical/test.txt', 'w') as file:
        file.write(text)

In [163]:
"""def createTest():
    with open('./Technical/test.txt', 'w') as file:
        for i in docs1:
            file.write(docs1[i])
            file.write('\n')"""

"def createTest():\n    with open('./Technical/test.txt', 'w') as file:\n        for i in docs1:\n            file.write(docs1[i])\n            file.write('\n')"

In [164]:
def clearTrain():
    for num, name in enumerate(os.listdir("./Dirty/Discussions/")):
        with open("./Dirty/Discussions/"+name, 'r') as doc:
            soup = BeautifulSoup(doc.read(), 'html.parser')
            with open("./Clean/Discussions/"+str(num)+'.txt', 'w') as clean:
                clean.write(' '.join(soup.getText(' ').strip().split()))

In [165]:
def createTrainfromDiss():
    text = ''
    for name in os.listdir("./Clean/Discussions/"):
        try:
            with open("./Clean/Discussions/"+name, 'r') as doc:
                #docs[name] = ""
                for word in re.findall(AllowedWords, doc.read().lower()):
                    normal = wordNormal.normal_forms(word)[0]
                    #docs[name] += str(normal)
                    #docs[name] += ' '
                    text += str(normal) + ' '
                text += '\n'
        except:
            print("haha")
    with open('./Technical/train.txt', 'w') as file:
        file.write(text)

In [166]:
"""def createTrain():
    with open('./Technical/ttest.txt', 'w') as file:
        for i in docs:
            file.write(' '.join(docs[i].strip().split()))
            file.write('\n')"""

"def createTrain():\n    with open('./Technical/ttest.txt', 'w') as file:\n        for i in docs:\n            file.write(' '.join(docs[i].strip().split()))\n            file.write('\n')"

In [167]:
cleanTest()
clearTrain()

In [168]:
createTrainfromDiss()
createTestfromData()

In [169]:
#def trainModel():
command = """./sent2vec/fasttext sent2vec -input \
    ./Technical/train.txt -output ./Model/{} -minCount 2 -dim 700\
        -epoch 9 -lr 0.15 -wordNgrams 2 -loss ns -neg 10 -thread 2\
            -t 0.000005 -dropoutK 4 -minCountLabel 10""".format(MODEL_NAME)
os.system(command)
#ToDo mb os.popen?

0

In [170]:
#def getDocsVectors():
command = """./sent2vec/fasttext print-sentence-vectors ./Model/{} < ./Technical/test.txt""".format(MODEL_NAME)
output = os.popen(command).readlines()
#answer = [i for i in output.strip().split('\n')] #ToDo mb abother split?
# This is N-ish vector ['1,2,3,4,4,5', '12,4,5,5,5,5,5', ...]

In [171]:
#def ClearVectors():
clearVec = dict()
with open('./Technical/test.txt', 'r') as test:
    text = test.readlines()
    for num, i in enumerate(output):
        clearVec[(text[num].strip())] = [float(j) for j in i.strip().split()]
        #dict like {'text': 'srting-list vector ['1,3,4,5,6,6']} <- first
        #dict like {'text': [1,2,3,4,5,7,8,]} <- now

#ToDo mb return empty dict/set/list?

In [172]:
#def getGoodSearch(search):
goodSearch = ''
search = ' '.join(search.split()).lower()
for word in re.findall(AllowedWords, search):
    goodSearch += str(word) + ' '
    #ToDo is it necessary?
    normal = wordNormal.normal_forms(word)[0]
    #ToDo mb put 2 normal forms? [0] and [1]??
    if normal not in stop_words:
        #ToDo stop words??
        goodSearch += str(normal) + ' '
with open('./Technical/input.txt', 'w') as inp:
    inp.write(goodSearch)

In [173]:
#ToDo mb put in not in file but in console text itself?

#def makeSearch(search):
command = """./sent2vec/fasttext print-sentence-vectors ./Model/{} < ./Technical/input.txt""".format(MODEL_NAME)
asearch = os.popen(command).read()
searchVec = [float(i) for i in asearch.strip().split()]


In [184]:
#def findClosest(clearVec, searchVec):
results = dict()
for num, i in enumerate(clearVec):
    if i == '':
        continue
    results[num] = 1 - cosine(clearVec[i], searchVec)
sortedResults = sorted(results, key=lambda x: results[x], reverse=True)


In [185]:
print(results)

{0: 0.30481501575310821, 1: 0.20738449401264059, 2: 0.12266420284670743, 3: -0.054052494291593867, 4: 0.12096538319444661, 5: 0.082486299328789703, 6: 0.02654543170229573, 7: 0.10365444136872759, 8: 0.13000318416672907, 9: 0.075630270677989553, 10: 0.076748639069064817, 11: -0.030492336530628883, 12: 0.042202440057955148, 13: 0.11088984710211658, 14: 0.12354955945355484, 15: 0.028003580979760856, 16: -0.021523869161895304, 17: 0.061988044710401979, 18: 0.10410834224897303, 19: 0.095763569930699788, 20: 0.13411265555987328, 21: 0.095095189403131664, 22: 0.062978945587391477, 23: 0.039993723073082688, 24: 0.031559117317904173, 25: 0.087686951720215012, 26: 0.040350318143373198, 27: 0.061308948383725692, 28: 0.10484961032997775, 29: 0.046753103972888899, 30: 0.09900457659286821, 31: 0.12764243091732275, 32: 0.13372631847315786, 33: 0.039818325546947086, 34: 0.14813834370491419, 35: 0.099640640871473396, 36: 0.14607135623361989, 37: 0.13569079949432461, 38: 0.086455002115053192, 39: 0.1094

In [186]:
sortedResults

[0,
 56,
 1,
 76,
 43,
 62,
 55,
 67,
 34,
 36,
 75,
 58,
 37,
 20,
 32,
 69,
 70,
 8,
 73,
 41,
 31,
 14,
 2,
 4,
 71,
 72,
 63,
 47,
 13,
 39,
 28,
 18,
 7,
 74,
 35,
 30,
 66,
 19,
 21,
 42,
 59,
 25,
 38,
 61,
 5,
 45,
 10,
 9,
 57,
 49,
 22,
 17,
 68,
 27,
 44,
 54,
 53,
 50,
 29,
 48,
 12,
 26,
 23,
 33,
 51,
 24,
 15,
 6,
 52,
 40,
 46,
 60,
 16,
 11,
 65,
 3]